In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
if not os.path.isdir("island_activity_files"):
    os.mkdir("island_activity_files")
from i2x.der_hca import hca, islands as isl, PlotUtils as pltutl
import numpy as np
import pandas as pd

>**Note about this notebook**<br>
>This notebook is a bit different from the others in this bootcamp and requires circling back a few times to the same cell.
>It is recommended to read once _through_ the [discussion section](#discussion) to get an overview, before diving into execution.
 
# Islanding Considerations in DER Integration
A common sticking point around DER integration on distribution feeders is focused on unintentional islands.
The basic concern is that if a fault occurs on the feeder, the breaker at the distribution substation opens to isolate the fault, but the DER on the feeder fails to detect the island and continues to feed the fault, at risk to both equipment and people.

The [i2X solution e-Xchange](https://www.energy.gov/eere/i2x/i2x-solution-e-xchanges) _Distribution System Protection with High DER Adoption Levels_ (Grid Engineering Practices & Standards) discussed this topic on May 3rd, 2023; the event was [recorded](https://youtu.be/haGZQfdPp1E) and is recommended viewing material for this exercise.
It discusses, among many other things, the use of the [Sandia Screen](https://www.osti.gov/biblio/1039001) and other alternatives to Direct Transfer Trip (DTT), which is an expensive and complex communications based solution.

The purpose of this exercise is to show how islanding consideration _could_ be incorporated into hosting capacity analysis, and how the use of time-series based methods can provide new ways of thinking about screening for the problem.
This is _not_ intended to be taken as _the_ solution.

## Approach to screening for potential islanding
The basic concept put forth in the [Sandia Screen](https://www.osti.gov/biblio/1039001) is that for an island to occur there needs to be both a real and reactive power balance at the time of the fault.
Building on this idea in the simulations, we monitor the flow on all reclosers in the system (this could be extended to other switching/protection devices if desired).
These devices are used to segment the feeder into components that are the potential islands.
The net flow in/out of each one of these is monitored via the following set of tests:
The following test is then carried out:
1. Is the potential island always importing or exporting _real power_?
	* Yes -> test is passed
	* No  -> proceed to step 2
2. Is the ratio of minimum output to maximum output above a certain level? Idea here is that the larger this ratio, the more abrupt the transition between importing to exporting and the less likely the load balance condition is met for any meaningful amount of time.
	* Yes -> test is passed
	* No  -> proceed to step 3
3. Repeat steps 1 and 2 for _reactive power_

## Visualizing components
The code to visualize the separable components of the feeders is provided in the auxiliary [feederplots notebook](./feederplots.ipynb).
The feeder plots themselves are available in the [feederplots](./feederplots/) folder.
The rest of the exercise will occasionally refer to a component number, the figures ending with `Comp.html` can help clarify what is referred to with each component.

# Utility Functions

>**Tip**<br>
>Feel free to skim or skip this section and come back to it if/when a more detailed dive into implementation is desired. However,<br>
&emsp;&emsp;**Make sure to run all code cells so the functions are defined!!!**

The following are a set of utility functions that will be used in the subsequent simulations.
Since our goal in this exercise is to bring about a situation where the anti-islanding constraint is binding, we would like a bit more control over which buses and capacities are sampled. The following functions do just that.


In [ ]:
def sample_bus(feeder:hca.HCA, comps:list):
    """This function samples feeder buses just like the hca routine
    but adds the flexibility of limiting the component the buses
    may belong to
    """
    buslist = [b for b in list(feeder.graph_dirs["bus3phase"]) + feeder.visited_buses if feeder.G.nodes[b]["comp"] in comps if b not in feeder.exauhsted_buses["pv"]]
    return feeder.sample_buslist(buslist)

def sample_pv(feeder:hca.HCA, min:int, max:int):
    """Similar to the internal function of the HCA object but setup here to allow
    sampling larger capacity more consistently
    """
    kw = float(feeder.random_state.randint(min, max+1))
    return {"kw": kw, "kva": kw/0.8}

The next set of functions are copied and adapted from the [voltage difference activity](./vdiff_activity.ipynb).
`get_upgrade_path` is used to find a path from the furthest node violating a particular metric to the source bus.
This path can then be used for conductor upgrades, which is exactly what the `upgrade_until_no_violation` function does.

In [ ]:
#########################################################################
####### these are some functions to get the path and perform the upgrades
#########################################################################
def get_upgrade_path(feeder:hca.HCA, voltagemetrics=["vdiff"]):
    """get the path from the furthest location from the source bus where
    a voltage difference problem occurs, to the source bus
    """
    ### collect locations where voltage difference is violated
    buses = set()
    if "vdiff" in voltagemetrics:
        buses.update(set(feeder.metrics.get_vdiff_locations()["v"].keys()))
    if "vmax" in voltagemetrics:
        buses.update(set(feeder.metrics.get_volt_max_buses().keys()))
        
    
    if not buses:
        return []
    
    sources = []
    upgrade_paths = []
    path_lengths = []
    ### for each location with a violation, find the path to the source bus
    for b in buses:
        nearest_source, path2source = isl.get_nearest_source(feeder.G.to_undirected(), b)
        sources.append(nearest_source)
        upgrade_paths.append(path2source)
        path_lengths.append(len(path2source))
    ### select the maximum distance to the source bus.
    idx = np.argmax(path_lengths)
    return upgrade_paths[idx]

def upgrade_until_no_violation(feeder:hca.HCA, upgrade_path:list, 
                               limit=None, err_threshold=0.2,
                               multiupgrade=True):
    """Iterate over the path to the source and upgrade the conductor 
    until no violations
    """
    if not upgrade_path:
        return
    cnt = 0
    err0 = None
    for u,v in zip(upgrade_path[1:], upgrade_path[:-1]):
        if (limit is not None) and (cnt >= limit):
            feeder.logger.info(f"Reached limited of {limit} upgrades. Stopping")
            break
        eclass = feeder.G.edges[u,v]["eclass"]
        ename = feeder.G.edges[u,v]["ename"]
        if eclass.lower() == "line":
            if not multiupgrade:
                if feeder.get_data("upgrades", "line", ename)[0] is not None:
                    # segment already updated
                    continue
            feeder.upgrade_line(ename)
            cnt += 1

        ## run dss
        feeder.reset_dss(clear_changes=False)
        feeder.rundss()
        if not feeder.lastres["converged"]:
            raise ValueError("Open DSS did not converge")
        
        feeder.metrics.load_res(feeder.lastres)
        feeder.metrics.calc_metrics()
        
        ## check if there are any violations
        if feeder.metrics.violation_count == 0:
            break
        else:
            err_new = feeder.metrics.violation["voltage"]["vdiff"]
        if err0 is not None:
            if err_new - err0 > err_threshold:
                feeder.logger.info(f"Vdiff violation got worse by {err_new - err0:0.2f}%. Stopping upgrades.")
                break
                
        err0 = err_new


The following two functions are used to adjust transformer taps and voltage regulator settings in the event of over-voltage violations (as well as under-voltage violations, technically).
Note that these functions modify _all_ transformers/regulators in the `upgrade_path`, returned from the `get_upgrade_path` function.

In [ ]:
def change_transformer_tap(feeder:hca.HCA, upgrade_path:list, tapchange:int):
    """Change the transformer tap setting by tapchange 
    for any transformer found along the upgrade_path

    returns True if any transformer was updated
    """
    out = False
    for u,v in zip(upgrade_path[1:], upgrade_path[:-1]):
        eclass = feeder.G.edges[u,v]["eclass"]
        ename = feeder.G.edges[u,v]["ename"]
        if eclass.lower() == "transformer":
            feeder.change_xfrm_tap(ename, tapchange)
            out = True
    return out

def change_regulator_vreg(feeder:hca.HCA, upgrade_path:list, vregchange:int):
    """Change the regulation voltage (behind PT) for any regulator 
    found along the upgrade_path
    
    returns True if any regulator was updated
    """
    out = False
    for u,v in zip(upgrade_path[1:], upgrade_path[:-1]):
        eclass = feeder.G.edges[u,v]["eclass"]
        ename = feeder.G.edges[u,v]["ename"]
        if eclass.lower() == "regulator":
            feeder.change_regulator_vreg(ename, vregchange)
            out = True
    return out

Finally, since we'll be trying to overcome several different kinds of violations, the function `check_violations` helps to quickly retest what is currently binding.

In [ ]:
def check_violations(feeder:hca.HCA):
    """rerun OpenDSS (presumably after some upgrades/updates)
    and reevaluate the metrics
    """
    feeder.reset_dss(clear_changes=False)
    feeder.rundss()
    if not feeder.lastres["converged"]:
        raise ValueError("Open DSS did not converge")


    feeder.metrics.load_res(feeder.lastres)
    feeder.metrics.calc_metrics()

    if feeder.metrics.violation_count > 0:
        hca.print_config(feeder.metrics.violation, printf=feeder.logger.info, title="Violations")
    else:
        feeder.logger.info("\nNo more violations!\n")

## Inverter Controls
As a reminder from the [voltage difference exercise](./vdiff_activity.ipynb), the available inverter control choices are:
```
'CONSTANT_PF', 'VOLT_WATT', 'VOLT_VAR_CATA', 'VOLT_VAR_CATB', 'VOLT_VAR_AVR', 'VOLT_VAR_VOLT_WATT', 'VOLT_VAR_14H'
```
In the cell below, change the choices to see the different curves.

In [ ]:
inverter_choice="VOLT_VAR_CATB"
pltutl.inverter_control_plot(inverter_choice)

## Simulation
### Description
The following simulation adds resources _only_ in component 0 (see [visualizing components section](#visualizing-components)), with the goal of pushing the simulation towards a condition where real and reactive power would balance and the islanding test would trigger a violation.

To achieve this, we add fairly large resources and address the resulting upgrades as needed.
We use a clear day profile for the solar generation to avoid the $\Delta V$ criterion that would otherwise dominate.

Finally, we want to investigate what impact inverter functions have on the result.

### Step 1: Choose inverter function and initialize
Select one of the [available inverter functions](#inverter-controls) and initialize the feeder object.


In [ ]:
inverter_choice = "VOLT_VAR_CATB"
logger_header = f"\n************* Island Activity (invmode={inverter_choice}) ******************\n"
feeder = hca.HCA("./configs/island_activity.json", 
                 logger_heading=logger_header)
feeder.inputs["invmode"] = inverter_choice
# feeder.inputs["debug_output"] = True
feeder.runbase()

### Step 2: Add large resources until violation
Instead of sampling like in other exercises, we make sure to:
1. Only sample in component 0 of the feeder (`sample_bus` function)
2. Sample between 1 and 10 MW capacities (`sample_pv` function)

>_Note:_ The `large_sample` flag is there because after several resources are added the power flow results are sometimes non-converged, even though OpenDSS technically solves. Basically, a low voltage solution is returned. In this situation, we undo the last addition and reduce the sample size to between 1 and 2 MW.

In [ ]:
large_sample = True
while True:
    bus = sample_bus(feeder, [0]) #sample only from component 0 (bottom of feeder)
    if large_sample:
        Sij = sample_pv(feeder, 1000, 10000) # sample between 1 MW and 10 MW
    else:
        Sij = sample_pv(feeder, 1000, 2000)
    feeder.hca_round("pv", bus=bus, Sij=Sij, allow_violations=True)
    if feeder.metrics.violation_count > 0:
        feeder.save(f"./island_activity_files/island_activity_{inverter_choice}_{feeder.cnt}_violation.pkl")
        if feeder.metrics.vdiff > 50:
            feeder.logger.info(f"\nUndoing round {feeder.cnt}: vdiff = {feeder.metrics.vdiff:0.2f}%")
            feeder.undo_hca_round("pv", feeder.visited_buses[-1], feeder.cnt)
            feeder.cnt -= 1

            large_sample = False
        else:
            break
    else:
        ## save for possible re-wind
        feeder.save(f"./island_activity_files/island_activity_{inverter_choice}_{feeder.cnt}_pre.pkl")

### Step 3: Resolve violations
Since violations occurred and we are trying to get to an islanding violation, we perform updates to get the system back to compliance.
That is, if the binding violation is _not_ `island_pq`, we perform upgrades.
The types of violations that are expected are:
* Over- and Under-Voltage;
* Thermal emergency and normal rating violations.

#### Thermal upgrades
First, the thermal violations are resolved by upgrading the overloaded conductors, similar to the the process in the [voltage difference exercise](./vdiff_activity.ipynb).

>**Tip** If you'd like to see the violations on the feeder, plot them first in the [thermal violation plotting section](#thermal-violation-plotting), and then return here.

In [ ]:
#### thermal updates
feeder = hca.HCA(f"./island_activity_files/island_activity_{inverter_choice}_{feeder.cnt}_violation.pkl",
                 reload=True, reload_filemode="a",
                 logger_heading=f"\n********* Reloading pre thermal upgrades (cnt={feeder.cnt}) *********\n")
feeder.metrics.load_res(feeder.lastres)
feeder.metrics.calc_metrics()
if np.any(["thermal" in v for v in feeder.metrics.last_violation_list]):
    ## resolve thermal violations
    for typ, names in feeder.metrics.get_thermal_branches().items():
        for name in names:
            if typ.lower() == 'line':
                feeder.upgrade_line(name)
            elif typ.lower() == "transformer":
                feeder.upgrade_xfrm(name)
    check_violations(feeder)
feeder.save(f"./island_activity_files/island_activity_{inverter_choice}_{feeder.cnt}_thermal.pkl")

#### Over-Voltage Violations
We tackle voltage violations in two main ways:
1. For nodes downstream from the voltage regulator (see diagrams in [`feederplots` folder](./feederplots/)) we can change the regulator voltage setting.
2. For all other nodes we can tweak the tap setting on the substation transformer.

The following code cell prints the bus and maximum voltage for all monitored buses that violate the voltage upper limit.

Take a look at where these are on the feeder.
Would you adjust the regulator setting or the substation transformer?

>**Tip** You can also plot some of the actual voltage traces in the [voltage min/max plotting section](#voltage-minmax-plotting), and then return here.

In [ ]:
### voltage updates
feeder = hca.HCA(f"./island_activity_files/island_activity_{inverter_choice}_{feeder.cnt}_thermal.pkl",
                 reload=True, reload_filemode="a",
                 logger_heading=f"\n********* Reloading post thermal, pre voltage upgrades (cnt={feeder.cnt})*********\n")
check_violations(feeder)
print("Maximum voltage violation locations:")
display(pd.DataFrame(pd.Series({k: v.max() for k, v in feeder.metrics.get_volt_max_buses().items()}, name="max voltage [p.u.]")))


In [ ]:
### get a path from all maximum voltage violating locations to the substation
upgrade_path = get_upgrade_path(feeder, voltagemetrics=["vmax"])

### try addressing via regulator change
# if all buses are upstream from transformer this will not do anything
if change_regulator_vreg(feeder, upgrade_path, -3): #-3
    check_violations(feeder)

### if no change and/or still violation, adjust the substation transformer taps
if feeder.metrics.violation_count:
    if change_transformer_tap(feeder, upgrade_path, 4): #4
        check_violations(feeder)

feeder.save(f"./island_activity_files/island_activity_{inverter_choice}_{feeder.cnt}_volt.pkl")

### Step 4: Repeat
At this point we go back to [Step 2](#step-2-add-large-resources-until-violation) to add another resource.
(clicking on the link should jump you right back there.)

We continue iterating from [Step 2](#step-2-add-large-resources-until-violation) to this point until either one of two things happen:
1. The _only_ binding violation is the islanding violation `island_pq`
2. We are out of good options for upgrading the feeder to accommodate the added resources without triggering violations.

You are encouraged to occasionally utilize the plotting functions in the [Plots Section](#plotting) to visualize the violations at every step.

# Discussion
Here is a description of the expected sequence of events along with some observations.
## With Inverter Mode `VOLT_VAR_CATB`
>**IMPORTANT:**<br>
>Make sure to begin this sequence by setting `inverter_choice="VOLT_VAR_CATB"` in [Step 1](#step-1-choose-inverter-function-and-initialize).
### Sequence of Events
* On the **first (1)** iteration 
    * Several thermal violations are remedied.
    * Adjustment of the voltage regulator is sufficient to resolve the over-voltage violation
* On the **second (2)** iteration
    * Several more thermal violations are remedied.
    * Following these, the only violation remaining is the islanding violation. Run the [island plot code cell](#plotting)

### Observations
Even though the islanding test is binding in the end, a closer look at the results reveals how this is still just a screen.
Namely, while both component 0 (bottom of the feeder) and component 2 (exchange with substation) have a $P$ and a $Q$ zero crossing, the two don't actually coincide exactly in time.
This fits with some of the observations mentioned in the [i2X Solution e-Xchange](https://www.energy.gov/eere/i2x/i2x-solution-e-xchanges) on the subject, where only a relatively small fraction of plants failing an anti-islanding screen actually display anti-islanding issues upon further study and therefore require DTT.

## With Inverter Mode `CONSTANT_PF`
>**IMPORTANT:**<br>
>Make sure to begin this sequence by setting `inverter_choice="CONSTANT_PF"` in [Step 1](#step-1-choose-inverter-function-and-initialize).
### Sequence of Events
* On the **first (1)** iteration
    * Several thermal violations are remedied.
    * Adjustment of the voltage regulator is sufficient to resolve the over-voltage violation
* On the **second (2)** iteration
    * Several more thermal violations are remedied, following which both a voltage maximum and the islanding constraints are being violated.
    * This time the over-voltage violations are upstream of the regulator and the substation transformer tabs are adjusted. This adjustment not only resolves the over-voltage problem, but the island constraint is also lifted.
* On the **third (3)** iteration
    * A thermal violation is remedied.
    * The over-voltage violation is now on both sides of the regulator, so both voltage regulator and transformer taps are adjusted.
    * An under-voltage violation now crops up

Feel free to play around more with the settings and see if you can resolve the issue.
For example, you can restart at the [Over-Voltage Violations](#over-voltage-violations) section, which reloads the feeder prior to the last modificaitions, and try to change the number of taps the substation transformer will be shifted.  
There are also example cells in the [Backup](#backup) section that perform further conductor upgrades.
Utilize the [plots](#voltage-minmax-plotting) to try get a feel for where, when, and to what extent the voltage violations are occurring.

### Observations
While the violations _may_ be resolvable with sufficient upgrades on the third iteration, the more important aspect of this exercise is that the anti-islanding constraint does not play a decisive role in limiting capacity with this control mode.

## Comparison
Compare the plot for component 0, $P$ and $Q$ for the case with `VOLT_VAR_CATB` vs. `CONSTANT_PF`.
With `VOLT_VAR_CATB`, when generation increases during the middle of the day, the reactive power moves in the opposite direction to help steady the voltage.
As a result, a change in direction occurs with both real and reactive power, leading to the possibility of unintentional load balance.
With `CONSTANT_PF`, on the other hand, there is far less variation in the reactive power, and as a result the change in direction does not occur.

# Plotting
## Islanding Plot
At any point during the exercise, you can run the cell below to create a plot of feeder component import/export to try to understand why the metric is behaving the way it is.
Definitely run this if the only active violation is the islanding one, but occasionally looking at this even if it is not binding can be helpful.

In [ ]:
# Give the plot a title that will have some meaning for you
title = f"{inverter_choice} (cnt={feeder.cnt}) islanding"

filebase = f"./island_activity_files/island_activity_{inverter_choice}_{feeder.cnt}_island"
pltutl.island_plots(feeder, filebase, plot_feeder=False, 
                    title=title,
                    include_plotlyjs='cdn', auto_open=True)

## Voltage Min/Max Plotting
Use the code cell below to plot maximum and minimum voltage violations.
Note that unlike the island plots, this will not produce a meaningful plot if there is not active violation.

In [ ]:
title = f"{inverter_choice} (cnt={feeder.cnt}) over- and under-voltage"

filebase = f"./island_activity_files/island_activity_{inverter_choice}_{feeder.cnt}_vmaxmin"
pltutl.vmaxmin_plot(feeder, filebase, plot_feeder=True, 
                    title=title,
                    include_plotlyjs='cdn', auto_open=True)

## Thermal Violation Plotting
Use the code cell below to plot the thermal violation on the feeder.
Like the voltage plots, this only produces meaningful results if there are active violations, otherwise it just produces a feeder plot.

In [ ]:
title = f"{inverter_choice} (cnt={feeder.cnt}) thermal violations"

filebase = f"./island_activity_files/island_activity_{inverter_choice}_{feeder.cnt}_thermal"
pltutl.thermal_plot(feeder, filebase,
                    title=title,
                    include_plotlyjs='cdn', auto_open=True)

# Backup
The code cells below might come in useful so they are kept here as backup.

**IMPORTANT:**<br>
Order of execution matters. Simply evaluating any of these cells may not work. They only make sense in particular contexts.

In [ ]:
### address voltage violations
## get upgrade path
upgrade_path = get_upgrade_path(feeder, voltagemetrics=["vdiff", "vmax"])
feeder.logger.info(f"Furthest violation from substation on bus {upgrade_path[0]}")

### Perform the upgrades
### assumption: were not going to upgrade segments twice
upgrade_until_no_violation(feeder, upgrade_path, limit=None, multiupgrade=False)
check_violations(feeder)

In [ ]:
### address voltage violations
## get upgrade path
feeder.cnt += 1
upgrade_path = get_upgrade_path(feeder, voltagemetrics=["vdiff", "vmax"])
feeder.logger.info(f"Furthest violation from substation on bus {upgrade_path[0]}")

### Perform the upgrades
### assumption: were not going to upgrade segments twice
upgrade_until_no_violation(feeder, upgrade_path, err_threshold=0.5,
                           limit=None, multiupgrade=True)
check_violations(feeder)

In [ ]:
## verify that an inverter control has indeed been applied in dss
feeder.verify_inverter_mode(verbose=True)

In [ ]:
## List all line upgrades
pd.concat([
    feeder.get_data("upgrades", "line", cnt=i) for i in range(feeder.cnt,0,-1)
    ], axis=0).sort_values("cnt").groupby(level=0).agg({
        "old": "first", "new": "last", 
        "length": "sum", "cost": "sum", 
        "cnt": "count"
        })

In [ ]:
## check regulator status
idx = feeder.dss.regcontrols.first()
print(feeder.dss.regcontrols.name)
print(feeder.dss.regcontrols.forward_vreg)
print(feeder.dss.regcontrols.reverse_vreg)